
<a id='harrison-kreps'></a>
<div id="qe-notebook-header" style="text-align:right;">
        <a href="https://quantecon.org/" title="quantecon.org">
                <img style="width:250px;display:inline;" src="https://assets.quantecon.org/img/qe-menubar-logo.svg" alt="QuantEcon">
        </a>
</div>

# Precificação de Ativos III: Mercados Incompletos


<a id='index-0'></a>

## Sumário

- [Precificação de Ativos III: Mercados Incompletos](#Precificação-de-Ativos-III:-Mercados-Incompletos)  
  - [Resumo](#Resumo)  
  - [Estrutura do Modelo](#Estrutura-do-Modelo)  
  - [Resolvendo o Modelo](#Resolvendo-o-Modelo)  
  - [Exercícios](#Exercícios)  
  - [Soluções](#Soluções)  

> *Devidamente traduzido, revisado e adaptado do [QuantEcon](https://julia.quantecon.org/) pelos bolsistas CNPq Pedro Luiz H. Furtado e Jonas Aragão M. Corpes, sob a supervisão do Prof. Christiano Penna, do CAEN/UFC.*

## Resumo

Esta palestra descreve uma versão de um modelo de Harrison e Kreps [HK78](https://julia.quantecon.org/../zreferences.html#harrkreps1978).

O modelo determina o preço de um ativo que gera dividendos que é negociado por dois tipos de investidores interessados.

O modelo apresenta:

- Crenças heterogêneas;
- Mercados incompletos
- Restrições curtas de vendas e possivelmente $ \ldots $
- (Alavancar) limites à capacidade de um investidor tomar empréstimos para financiar compras de um ativo arriscado  

### Referências

Antes de ler esta aula, você pode revisar nossas aulas sobre:

- [Cadeias de Markov](https://julia.quantecon.org/../tools_and_techniques/finite_markov.html)
- [Preços de ativos com espaço de estado finito](https://julia.quantecon.org/markov_asset.html)  

### Bolhas

Os economistas diferem em como definem uma "bolha".

O modelo de Harrison-Kreps ilustra a seguinte noção de bolha que atrai muitos economistas:

> *Um componente do preço de um ativo pode ser interpretado como uma bolha quando todos os investidores concordam que o preço atual do ativo excede o que eles acreditam que o fluxo de dividendos subjacente do ativo justifica.*

### Configuração

In [1]:
using InstantiateFromURL
github_project("QuantEcon/quantecon-notebooks-julia", version = "0.4.0")

In [2]:
using LinearAlgebra, Statistics

## Estrutura do Modelo

O modelo simplificado ignora alterações na distribuição da riqueza
entre investidores com diferentes crenças sobre os fundamentos que determinam
pagamentos de ativos.

Há um número fixo de $ A $ em ações de um ativo.

Cada ação autoriza seu proprietário a um fluxo de dividendos $ \{d_t \} $ governados por uma cadeia de Markov definida em um espaço de estado $ S \in \{0, 1 \} $.

O dividendo obedece

$$
d_t =
\begin{cases}
    0 & \text{ se } s_t = 0 \\
    1 & \text{ se } s_t = 1
\end{cases}
$$

O proprietário de uma ação no início do período $ t $ tem direito ao dividendo pago no período $ t $.

O proprietário da ação no início do período $ t $ também tem o direito de vender a ação a outro investidor durante o período $ t $.

Dois tipos $ h = a, b $ de investidores diferem apenas em suas crenças sobre uma matriz de transição de Markov $ P $ com elemento típico

$$
P(i,j) = \mathbb P\{s_{t+1} = j \mid s_t = i\}
$$

Investidores do tipo $ a $ acreditam na a matriz de transição é:

$$
P_a =
    \begin{bmatrix}
        \frac{1}{2} & \frac{1}{2} \\
        \frac{2}{3} & \frac{1}{3}
    \end{bmatrix}
$$

Já os investidores do tipo $ b $ acham que a matriz de transição é:

$$
P_b =
    \begin{bmatrix}
        \frac{2}{3} & \frac{1}{3} \\
        \frac{1}{4} & \frac{3}{4}
    \end{bmatrix}
$$

As distribuições estacionárias (ou seja, invariantes) dessas duas matrizes podem ser calculadas da seguinte forma:

In [3]:
using QuantEcon

qa = [1/2 1/2; 2/3 1/3]
qb = [2/3 1/3; 1/4 3/4]
mcA = MarkovChain(qa)
mcB = MarkovChain(qb)
stA = stationary_distributions(mcA)

1-element Array{Array{Float64,1},1}:
 [0.5714285714285715, 0.4285714285714286]

In [4]:
stB = stationary_distributions(mcB)

1-element Array{Array{Float64,1},1}:
 [0.42857142857142855, 0.5714285714285714]

A distribuição estacionária de $ P_a $ é aproximadamente $ \pi_A = \begin{bmatrix} .57 & .43 \end{bmatrix} $.

A distribuição estacionária de $ P_b $ é aproximadamente $ \pi_B = \begin{bmatrix} .43 & .57 \end{bmatrix} $.

### Direitos de propriedade

Um proprietário do ativo no final do período $ t $ tem direito ao dividendo no período $ t + 1 $ e também tem o direito de vender o ativo no período $ t + 1 $.

Ambos os tipos de investidores são neutros ao risco e ambos têm o mesmo fator de desconto fixo $ \beta \in (0,1) $.

Em nosso exemplo numérico, definiremos $ \beta = 0,75 $, assim como Harrison e Kreps.

Finalmente, estudaremos as consequências de duas suposições diferentes sobre o número de ações $ A $ em relação aos recursos que nossos dois tipos de investidores podem investir nas ações.

1. Ambos os tipos de investidores têm recursos suficientes (riqueza ou capacidade de contrair empréstimos) para que possam comprar todo o estoque disponível do ativo <sup><a href=#f1 id=f1-link>[1]</a></sup>.
2. Nenhum tipo de investidor possui recursos suficientes para comprar todo o estoque.


O caso 1 é o caso estudado em Harrison e Kreps.

No caso 2, ambos os tipos de investidor sempre mantêm pelo menos parte do ativo.

### Proibição de venda à descoberto

Não são permitidas vendas a descoberto.

Isso importa porque limita os pessimistas de expressar suas opiniões:

- Eles podem expressar suas opiniões vendendo suas ações, e
- Eles não podem expressar seu pessimismo mais alto com “ações de fabricação” artificialmente - isto é, eles não podem emprestar ações de investidores mais otimistas e vendê-las imediatamente.  

### Otimismo e Pessimismo

As especificações acima das matrizes de transição percebidas $ P_a $ e $ P_b $, obtidas diretamente de Harrison e Kreps, constroem otimismo e pessimismo temporariamente alternados estocamente.

Lembre-se de que o estado $ 1 $ é o estado com altos dividendos.

- No estado $ 0 $, um agente do tipo $ a $ é mais otimista quanto aos dividendos do próximo período do que um agente do tipo $ b $.
- No estado $ 1 $, um agente do tipo $ b $ é mais otimista sobre o dividendo do próximo período.


No entanto, as distribuições estacionárias $ \pi_A = \begin {bmatrix} .57 & .43 \end {bmatrix} $ e $ \pi_B = \begin {bmatrix} .43 & .57 \end {bmatrix} $ nos dizem que um a pessoa do tipo $ B $ é mais otimista em relação ao processo de dividendos a longo prazo do que uma pessoa do tipo A.

As matrizes de transição para os investidores temporariamente otimistas e pessimistas são construídas da seguinte forma.

Investidores temporariamente otimistas (ou seja, o investidor com mais otimismo
crenças em cada estado) acreditam que a matriz de transição

$$
P_o =
    \begin{bmatrix}
        \frac{1}{2} & \frac{1}{2} \\
        \frac{1}{4} & \frac{3}{4}
    \end{bmatrix}
$$

Temporariamente, pessimistas acreditam que a matriz de transição será:

$$
P_p =
    \begin{bmatrix}
        \frac{1}{2} & \frac{1}{2} \\
        \frac{1}{4} & \frac{3}{4}
    \end{bmatrix}
$$

Voltaremos a essas matrizes e sua importância no exercício.

### Informação

Os investidores conhecem uma função de preço que mapeia o estado $ s_t $ em $ t $ para o preço de equilíbrio $ p (s_t) $ que prevalece nesse estado.

Essa função de preço é endógena e deve ser determinada abaixo.

Quando os investidores escolhem comprar ou vender o ativo por US $ t $, eles também sabem $ s_t $.


## Resolvendo o Modelo

Agora vamos resolver o modelo.

Isso equivale a determinar os preços de equilíbrio sob as diferentes especificações possíveis de crenças e restrições listadas acima.

Em particular, comparamos as funções de preço de equilíbrio sob a seguinte alternativa
suposições sobre crenças:

1. Existe apenas um tipo de agente, $ a $ ou $ b $.
1. Existem dois tipos de agentes diferenciados apenas por suas crenças. Cada tipo de agente tem recursos suficientes para comprar todo o ativo (configuração de Harrison e Kreps).
1. Existem dois tipos de agentes com crenças diferentes, mas devido à riqueza limitada e/ou alavancagem limitada, ambos os tipos de investidores mantêm o ativo a cada período.  

### Tabeka de resumo

A tabela a seguir apresenta um resumo dos resultados obtidos no restante da palestra
(você será solicitado a recriar a tabela em um exercício).

Ele registra implicações das especificações de Harrison e Kreps de $ P_a, P_b, \beta $.

|$ s_t $|0|1|
|:---------------------:|:----:|:----:|
|$ p_a $|1.33|1.22|
|$ p_b $|1.45|1.91|
|$ p_o $|1.85|2.08|
|$ p_p $|1|1|
|$ \hat{p}_a $|1.85|1.69|
|$ \hat{p}_b $|1.69|2.08|

Aqui:

- $ p_a $ é a função de preço de equilíbrio sob crenças homogêneas $ P_a $;
- $ p_b $ é a função de preço de equilíbrio sob crenças homogêneas $ P_b $;
- $ p_o $ é a função de preço de equilíbrio sob crenças heterogêneas com investidores marginais otimistas;
- $ p_p $ é a função de preço de equilíbrio sob crenças heterogêneas com investidores marginais pessimistas;
- $ \hat {p} _a $ é o tipo de valor $ a $ que os investidores estão dispostos a pagar pelo ativo, e
- $ \hat {p} _b $ é o tipo de valor $ b $ que os investidores estão dispostos a pagar pelo ativo.  


Explicaremos esses valores e como eles são calculados uma linha por vez.

### Preços de crença simples

Começaremos por precificar o ativo de acordo com crenças homogêneas.

(Este é o caso tratado [na aula](https://julia.quantecon.org/markov_asset.html) sobre preços de ativos com estados finitos de Markov)

Suponha que exista apenas um tipo de investidor, do tipo $ a $ ou $ b $, e que esse investidor sempre “precifica o ativo”.

Seja $ p_h = \begin {bmatrix} p_h (0) \cr p_h (1) \end {bmatrix} $ seja o vetor de preço de equilíbrio quando todos os investidores forem do tipo $ h $.

O preço hoje é igual ao valor descontado esperado dos dividendos de amanhã e ao preço do ativo de amanhã:

$$
p_h(s) = \beta \left( P_h(s,0) (0 + p_h(0)) + P_h(s,1) ( 1 + p_h(1)) \right), \quad s = 0, 1
$$

Essas equações implicam que o vetor de preço de equilíbrio é


<a id='equation-harrkrep1'></a>
$$
\begin{bmatrix} p_h(0) \cr p_h(1) \end{bmatrix}
= \beta [I - \beta P_h]^{-1} P_h \begin{bmatrix} 0 \cr 1 \end{bmatrix} \tag{1}
$$

As duas primeiras linhas da tabela relatam $ p_a (s) $ e $ p_b (s) $.

Aqui está uma função que pode ser usada para calcular esses valores:

In [5]:
using LinearAlgebra

function price_single_beliefs(transition, dividend_payoff;
                            β=.75)
    # Primeiro, computar as perças inversas
    imbq_inv = inv(I - β * transition)

    # Depois, computar os preços
    prices = β * ((imbq_inv * transition) * dividend_payoff)

    return prices
end

price_single_beliefs (generic function with 1 method)

#### Crença em preços simples como referências

Esses preços de equilíbrio sob crenças homogêneas são marcos importantes para a análise subsequente.

- $ p_h (s) $ diz o que o investidor $ h $ pensa ser o "valor fundamental" do ativo.
- Aqui "valor fundamental" significa o valor presente descontado esperado de dividendos futuros.


Compararemos esses valores fundamentais do ativo com os valores de equilíbrio quando os traders tiverem crenças diferentes.

### Precificação sob crenças heterogêneas

Existem vários casos a considerar.

A primeira é quando os dois tipos de agente têm riqueza suficiente para comprar todo o ativo.

Nesse caso, o investidor marginal que precifica o ativo é do tipo mais otimista, de modo que o preço de equilíbrio $ \bar p $ satisfaz a equação principal de Harrison e Kreps:


<a id='equation-hakr2'></a>
$$
\bar p(s) =
\beta
\max
\left\{
        P_a(s,0) \bar p(0) + P_a(s,1) ( 1 +  \bar p(1))
        ,\;
        P_b(s,0) \bar p(0) + P_b(s,1) ( 1 +  \bar p(1))
\right\} \tag{2}
$$

for $ s=0,1 $.

O investidor marginal que precifica o ativo no estado $ s $ é do tipo $ a $ se:

$$
P_a(s,0)  \bar p(0) + P_a(s,1) ( 1 +  \bar p(1)) >
P_b(s,0)  \bar p(0) + P_b(s,1) ( 1 +  \bar p(1))
$$

O investidor marginal é do tipo $ b $ se:

$$
P_a(s,1)  \bar p(0) + P_a(s,1) ( 1 +  \bar  p(1)) <
P_b(s,1)  \bar p(0) + P_b(s,1) ( 1 +  \bar  p(1))
$$

**Assim, o investidor marginal é do tipo otimista (temporariamente)**.

A equação [(2)](#equation-hakr2) é uma equação funcional que, como uma equação de Bellman, pode ser resolvida por

- Começar com um palpite para o vetor de preço $ \bar p $, e
- Ir iterando para a convergência no operador que mapeia um palpite $ \bar p ^ j $ em um palpite atualizado
   $ \bar p ^ {j + 1} $ definido pelo lado direito de [(2)](#equation-hakr2), a saber  



<a id='equation-harrkrep3'></a>
$$
\bar  p^{j+1}(s)
 = \beta \max
 \left\{
        P_a(s,0) \bar p^j(0) + P_a(s,1) ( 1 + \bar p^j(1))
        ,\;
        P_b(s,0) \bar p^j(0) + P_b(s,1) ( 1 + \bar p^j(1))
\right\} \tag{3}
$$

for $ s=0,1 $.

A terceira linha da tabela relata preços de equilíbrio que resolvem a equação funcional quando $ \beta = 0,75 $.

Aqui, o tipo otimista de $ s_ {t + 1} $ precifica o ativo no estado $ s_t $.

É instrutivo comparar esses preços com os preços de equilíbrio das economias de crenças homogêneas que resolvem sob as crenças $ P_a $ e $ P_b $.

Os preços de equilíbrio $ \bar p $ na economia de crenças heterogêneas excedem o que qualquer investidor em potencial considera como o valor fundamental do ativo em cada estado possível.

No entanto, a economia visita recorrentemente um estado que faz com que cada investidor deseje
comprar o ativo por mais do que ele acredita que seus dividendos futuros sejam
que vale a pena.

O motivo é que ele espera ter a opção de vender o ativo posteriormente para outro investidor que valorize o ativo mais do que ele.

- Investidores do tipo $ a $ estão dispostos a pagar o seguinte preço pelo ativo  


$$
\hat p_a(s) =
\begin{cases}
\bar p(0)  & \text{ if } s_t = 0 \\
\beta(P_a(1,0) \bar p(0) + P_a(1,1) ( 1 +  \bar p(1))) & \text{ if } s_t = 1
\end{cases}
$$

- Investidores do tipo $ b $ estão dispostos a pagar o seguinte preço pelo ativo


$$
\hat p_b(s) =
\begin{cases}
    \beta(P_b(0,0) \bar p(0) + P_b (0,1) ( 1 +  \bar p(1)))  & \text{ if } s_t = 0 \\
    \bar p(1)  & \text{ if } s_t =1
\end{cases}
$$

Evidentemente, $ \hat p_a (1) <\bar p (1) $ e $ \hat p_b (0) <\bar p (0) $.

Investidores do tipo $ a $ desejam vender o ativo no estado $ 1 $ enquanto investidores do tipo $ b $ desejam vendê-lo no estado $ 0 $.

- O ativo muda de mãos sempre que o estado muda de $ 0 $ para $ 1 $ ou de $ 1 $ para $ 0 $.
- As avaliações $ \hat p_a (s) $ e $ \hat p_b (s) $ são exibidas na quarta e quinta linhas da tabela.
- Até os investidores pessimistas que não compram o ativo acham que vale mais do que pensam que os dividendos futuros valem.


Aqui está o código para resolver $ \bar p $, $ \hat p_a $ e $ \hat p_b $ usando o método iterativo descrito acima:

In [6]:
function price_optimistic_beliefs(transitions,
                                  dividend_payoff;
                                  β=.75, max_iter=50000,
                                  tol=1e-16)

    # Vamos adivinhar um vetor de preço inicial de [0, 0]
    p_new = [0,0]
    p_old = [10.0,10.0]

    # Sabemos que este é um mapeamento de contração, para que possamos iterar para
    for i ∈ 1:max_iter
        p_old = p_new
        temp = [maximum((q * p_old) + (q * dividend_payoff))
                            for q in transitions]
        p_new = β * temp

        # Se conseguirmos convergir, saia do loop para
        if maximum(sqrt, ((p_new - p_old).^2)) < 1e-12
            break
        end
    end

    temp=[minimum((q * p_old) + (q * dividend_payoff)) for q in transitions]
    ptwiddle = β * temp

    phat_a = [p_new[1], ptwiddle[2]]
    phat_b = [ptwiddle[1], p_new[2]]

    return p_new, phat_a, phat_b
end

price_optimistic_beliefs (generic function with 1 method)

### Fundos insufucientes

Os resultados diferem quando o tipo mais otimista de investidor possui riqueza insuficiente - ou capacidade insuficiente para emprestar o suficiente - para manter todo o estoque do ativo.

Nesse caso, o preço do ativo deve ser ajustado para atrair investidores pessimistas.

Em vez da equação [(2)](#equation-hakr2), o preço de equilíbrio satisfaz


<a id='equation-harrkrep4'></a>
$$
\check p(s)
= \beta \min
\left\{
    P_a(s,1)  \check  p(0) + P_a(s,1) ( 1 +   \check  p(1)) ,\;
    P_b(s,1)  \check p(0) + P_b(s,1) ( 1 + \check p(1))
\right\} \tag{4}
$$

e o investidor marginal que precifica o ativo é sempre aquele que o valoriza *menos* do que o outro tipo.

Agora, o investidor marginal é sempre do tipo (temporariamente) pessimista.

Observe na sexta linha que o preço pessimista $ \underline  p $ é menor que os preços homogêneos de crença $ p_a $ e $ p_b $ nos dois estados.

Quando investidores pessimistas precificam o ativo de acordo com [(4)](#equation-harrkrep4), investidores otimistas pensam que o ativo está subvalorizado.

Se pudessem, investidores otimistas emprestariam de bom grado a taxa de juros bruta de um período $ \beta ^ {- 1} $ para comprar mais ativos.

Restrições implícitas à alavancagem os proíbem de fazê-lo.

Quando investidores otimistas precificam o ativo como na equação [(2)](#equation-hakr2), investidores pessimistas pensam que o ativo está supervalorizado e gostaria de vendê-lo a descoberto.

Restrições às vendas a descoberto impedem isso.

Aqui está o código para resolver $ \check p $ usando a iteração:

In [7]:
function price_pessimistic_beliefs(transitions,
                                   dividend_payoff;
                                   β=.75, max_iter=50000,
                                   tol=1e-16)
    # Vamos adivinhar um vetor de preço inicial de [0, 0]
    p_new = [0, 0]
    p_old = [10.0, 10.0]

    # Sabemos que este é um mapeamento de contração, para que possamos iterar para
    for i ∈ 1:max_iter
        p_old = p_new
        temp=[minimum((q * p_old) + (q* dividend_payoff)) for q in transitions]
        p_new = β * temp

        # Se conseguirmos convergir, saia do loop for
        if maximum(sqrt, ((p_new - p_old).^2)) < 1e-12
            break
        end
    end

    return p_new
end

price_pessimistic_beliefs (generic function with 1 method)

### Interpretação adicional

[Sch14](https://julia.quantecon.org/../zreferences.html#scheinkman2014) interpreta o modelo de Harrison-Kreps como um modelo de bolha - uma situação em que o preço de um ativo excede o que todo investidor pensa é merecido pelo fluxo de dividendos subjacente do ativo.

Scheinkman enfatiza esses recursos do modelo Harrison-Kreps:

- Em comparação com a configuração de crenças homogêneas que leva à fórmula de preços, ocorre um alto volume quando a fórmula de preços da Harrison-Kreps prevalece.


Os investidores do tipo $ a $ vendem todo o estoque do ativo para o tipo $ b $ de investidores sempre que o estado passar de $ s_t = 0 $ para $ s_t = 1 $.

Digite $ b $ investidores para vender o ativo para digitar $ a $ investidores sempre que o estado passar de $ s_t = 1 $ para $ s_t = 0 $.

Scheinkman toma isso como uma força do modelo, porque ele observa alto volume durante *bolhas famosas*.

- Se a *oferta* do ativo é aumentada suficientemente, tanto fisicamente (mais "casas" são construídas) ou artificialmente (formas são inventadas para vender "casas" a descoberto), as bolhas terminam quando a oferta cresce o suficiente para superar os recursos otimistas dos investidores para comprar o ativo.
- Se investidores otimistas financiarem compras por meio de empréstimos, o aperto nas restrições de alavancagem pode extinguir uma bolha.


Scheinkman extrai informações sobre os efeitos dos regulamentos financeiros nas bolhas.

Ele enfatiza como limitar as vendas a descoberto e alavancar a limitação têm efeitos opostos.

## Exercícios

### Exercício 1

Recrie a tabela de resumo usando as funções que criamos acima.

|$ s_t $|0|1|
|:---------------------:|:----:|:----:|
|$ p_a $|1.33|1.22|
|$ p_b $|1.45|1.91|
|$ p_o $|1.85|2.08|
|$ p_p $|1|1|
|$ \hat{p}_a $|1.85|1.69|
|$ \hat{p}_b $|1.69|2.08|

Primeiro, você precisará definir as matrizes de transição e o vetor de pagamento de dividendos.

## Soluções

### Exercício 1

Primeiro, obteremos vetores de preços de equilíbrio com crenças homogêneas, inclusive quando todos
investidores são otimistas ou pessimistas:

In [8]:
qa = [1/2 1/2; 2/3 1/3]     # Escreva matriz de transição a
qb = [2/3 1/3; 1/4 3/4]     # Escreva matriz de transição b
qopt = [1/2 1/2; 1/4 3/4]   # Matriz de transição do investidor otimista
qpess = [2/3 1/3; 2/3 1/3]  # Matriz de transição do investidor pessimista

dividendreturn = [0; 1]

transitions = [qa, qb, qopt, qpess]
labels = ["p_a", "p_b", "p_optimistic", "p_pessimistic"]

for (transition, label) in zip(transitions, labels)
    println(label)
    println(repeat("=", 20))
    s0, s1 = round.(price_single_beliefs(transition, dividendreturn), digits=2)
    println("State 0: $s0")
    println("State 1: $s1")
    println(repeat("-", 20))
end

p_a
State 0: 1.33
State 1: 1.22
--------------------
p_b
State 0: 1.45
State 1: 1.91
--------------------
p_optimistic
State 0: 1.85
State 1: 2.08
--------------------
p_pessimistic
State 0: 1.0
State 1: 1.0
--------------------


Usaremos a função price_optimistic_beliefs para encontrar o preço em
crenças heterogêneas.

In [9]:
opt_beliefs = price_optimistic_beliefs([qa, qb], dividendreturn)
labels = ["p_optimistic", "p_hat_a", "p_hat_b"]

for (p, label) ∈ zip(opt_beliefs, labels)
    println(label)
    println(repeat("=", 20))
    s0, s1 = round.(p, digits = 2)
    println("State 0: $s0")
    println("State 1: $s1")
    println(repeat("-", 20))
end

p_optimistic
State 0: 1.85
State 1: 2.08
--------------------
p_hat_a
State 0: 1.85
State 1: 1.69
--------------------
p_hat_b
State 0: 1.69
State 1: 2.08
--------------------


Observe que o preço de equilíbrio com crenças heterogêneas é igual ao preço sob crenças únicas
com investidores otimistas - isso ocorre porque o investidor marginal é do tipo temporariamente otimista.

**Notas de rodapé**

<p><a id=f1 href=#f1-link><strong>[1]</strong> </a> Supondo que os dois tipos de agente sempre tenham "bolsos suficientemente profundos" para comprar todo o ativo, o modelo tira a dinâmica da riqueza da mesa. O modelo Harrison-Kreps gera alto volume de negociação quando o estado muda de 0 para 1 ou de 1 para 0.